# Solver for TwentyLetters.com
by Trey Reynolds 02/22/22

Steps:

1. Download the Scrabble dictionary from some random github repo.
2. Pre-compute a sorted word data structure.
  * Pre-sorting the dictionary only happens once.
  * This gives a 2x-5x speed increase in the worst case search.
3. Compute the full list of all possible words sorted by score.
4. Recursively do the same procedure with remaining letters until either:
  * There are no remaining letters.
  * No words are found with the remaining letters.
5. If the recursion base case is met back up and try the next "final word".
  * There is a specified search depth of words per "level".
  * The default depth is 30 words.
6. Keep doing this until you run out of words, print only the best results.



In [22]:
from bisect import bisect_left
from itertools import combinations, product
import requests

In [ ]:
# Game Definitions
scores = {
  "a": 1,"b": 3,"c": 3,"d": 2,"e": 1,"f": 4,"g": 2,"h": 4,"i": 1,"j": 8,"k": 5,
  "l": 1,"m": 3,"n": 1,"o": 1,"p": 3,"q": 10,"r": 1,"s": 1,"t": 1,"u": 1,"v": 4,
  "w": 4,"x": 8,"y": 4,"z": 10
}
minimum_word_length = 3

1. Download the Scrabble dictionary from some random github repo.

In [23]:
url = "https://raw.githubusercontent.com/zeisler/scrabble/master/db/dictionary.csv"
r = requests.get(url)
f = open('scrabble_dictionary.txt','w')
f.write(r.text)

1916146

Pre-compute a sorted word data structure.
  * Pre-sorting the dictionary only happens once.
  * This gives a 2x-5x speed increase in the worst case search.

In [32]:
# Writing the dictionary out as all possible sorted anagrams speeds up search 5-10x
f = open('scrabble_dictionary.txt')
d = {}

for word in f:
  if len(word) >= minimum_word_length:
    word = word.strip()
    key = ''.join(sorted(word))
    if key in d:
      d[key].append(word)
    else:
      d[key] = [word]

f.close()
anagram_dictionary = [' '.join([key]+value) for key, value in d.items()]
anagram_dictionary.sort()
f = open('anagram_dictionary.txt','w')
f.write('\n'.join(anagram_dictionary))
f.close()

In [26]:
# Note the difference from scabble -- word length is a multiplier
def score_word(word):
  return sum([scores[c] for c in word]) * len(word)

In [27]:
# Load this so we don't have to do it every round
def load_anagram_dictionary():
  f = open('anagram_dictionary.txt','r')
  anagram_dictionary = f.read().split('\n')
  f.close()
  return anagram_dictionary

In [28]:
def findwords(letters, anagram_dictionary):
  letters = ''.join(sorted(letters))
  foundwords = []
  for i in range(2,len(letters)+1):
    for comb in combinations(letters,i):
      ana = ''.join(comb)
      j = bisect_left(anagram_dictionary, ana)
      if j == len(anagram_dictionary):
        continue
      words = anagram_dictionary[j].split()
      if words[0] == ana:
        foundwords.extend(words[1:])
  return foundwords

In [29]:
def remove_letters(remaining, word):
  new_remaining = remaining
  for character in word:
    new_remaining = new_remaining.replace(character, "", 1)
  return new_remaining

Compute the full list of all possible words sorted by score.
Recursively do the same procedure with remaining letters until either:
  * There are no remaining letters.
  * No words are found with the remaining letters.

In [30]:
# Cache for found words so we don't have to compute every time
found = {}

# Recursive function to search for the possible word combos from the indices passed
def find_combo(remaining, words, level, indices):
  if remaining == "":
    return words

  remaining = remove_letters(remaining, "".join(words))
  if not remaining in found:
    found[remaining] = sorted(set(findwords(remaining, anagram_dictionary)), key=lambda w: score_word(w), reverse=True)

  if len(found[remaining]) == 0:
    return words

  if indices[level] >= len(found[remaining]):
    return words

  words += [found[remaining][indices[level]]]

  return find_combo(remaining, words, level + 1, indices)

5. If the recursion base case is met back up and try the next "final word".
  * There is a specified search depth of words per "level".
  * The default depth is 30 words.  
6. Keep doing this until you run out of words, print only the best results.

In [31]:
board = "aaabghiiiilllnotttuy"
anagram_dictionary = load_anagram_dictionary()
search_depth = 40
best_score = 0

for w1, w2, w3, w4 in product(range(search_depth), repeat=4):
   result_words = find_combo(board, [], 0, [w1,w2,w3,w4,0,0])
   result_string = "".join(result_words)
   result_score = sum(score_word(r) for r in result_words)
   if len(result_string) == len(board) and best_score < result_score:
     best_score = result_score
     print(str(result_score) + " - " + " ".join(result_words))

241 - gullibility attain hao
290 - habitually litigation
